<a href="https://www.kaggle.com/code/edwinstanzah/eda-with-plotly-data-science-salaries?scriptVersionId=105787411" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Table of Contents

* [Question 1](#q1)
* [Question 2](#q2)
* [Question 3](#q3)
* [Question 4](#q3)
* [Question 5](#q3)

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# Plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [2]:
data = pd.read_csv('../input/data-science-job-salaries/ds_salaries.csv')

In [3]:
data.head()

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


In [4]:
# drop Unnamed:0
data.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           607 non-null    int64 
 1   experience_level    607 non-null    object
 2   employment_type     607 non-null    object
 3   job_title           607 non-null    object
 4   salary              607 non-null    int64 
 5   salary_currency     607 non-null    object
 6   salary_in_usd       607 non-null    int64 
 7   employee_residence  607 non-null    object
 8   remote_ratio        607 non-null    int64 
 9   company_location    607 non-null    object
 10  company_size        607 non-null    object
dtypes: int64(4), object(7)
memory usage: 52.3+ KB


The data has no missing values, which is great. There's a lot of direction I could take here. But first, let's formulate some questions. Maybe you, as someone who aspires to be a data scientist have similar questions. 

1. Do the salary of data scientist increase from 2020 to 2022?
2. What can you expect to be paid for a given experience level?
3. What are the job titles for different experience level? 
4. Does company location affect how much you can earn as a data scientist? 
5. Are bigger companies paying more than smaller companies?


# Question 1<a id="q1"></a>

<div style="padding:20px;color:black;margin:0;font-size:200%;text-align:center;display:fill;border-radius:5px;background-color:#74bde0;overflow:hidden;font-weight:500">Do the salary of data scientist increase from 2020 to 2022?</div>


In [6]:
data.employment_type.value_counts()

FT    588
PT     10
CT      5
FL      4
Name: employment_type, dtype: int64

* The vast majority of the data is for full-time data scientists, for consistency I will drop the other type of employment

In [7]:
# only choose Full Time data scientists
FT_data = data[data.employment_type == 'FT']

In [8]:
FT_data.work_year.value_counts()

2022    314
2021    206
2020     68
Name: work_year, dtype: int64

In [9]:
# check the distribution of pay based on year
fig1 = px.histogram(FT_data, 
                   x='salary_in_usd', 
                   color='work_year', 
                   nbins=100, 
                   barmode='overlay'
                   )
fig2 = px.box(FT_data,
              x='salary_in_usd',
              color='work_year',
              template='presentation'
             )
fig1.show()
fig2.show()


As you can see, there is a bit of a shift, particularly between 2021 and 2022 salaries where more people are being paid over 100k for the 2022 work_year. The median of the salaries has indeed increase over this 3-year period. 

# Question 2<a id="q2"></a>

<div style="padding:20px;color:black;margin:0;font-size:200%;text-align:center;display:fill;border-radius:5px;background-color:#74bde0;overflow:hidden;font-weight:500">What kind of salary can we expect given an experience level?</div>

In [10]:
fig1 = px.histogram(FT_data, 
                   x='salary_in_usd', 
                   color='experience_level', 
                   nbins=100, 
                   barmode='overlay'
                   )
fig2 = px.box(FT_data,
              x='salary_in_usd',
              color='experience_level'
             )
fig1.show()
fig2.show()

Indeed, as expected, there's a difference in pay based on experience level. They have overlapping salary ranges but in general it's quite obvious from the chart that more experienced data scientists are compensated more. 

# Question 3<a id="q3"></a>

<div style="padding:20px;color:black;margin:0;font-size:200%;text-align:center;display:fill;border-radius:5px;background-color:#74bde0;overflow:hidden;font-weight:500">What are the job titles for different experience level?</div>

In [11]:
fig = px.sunburst(
        FT_data,
        path=['experience_level', 'job_title'], 
        values='salary_in_usd',
        width=800,
        height=800,
        color_discrete_sequence=px.colors.qualitative.Pastel,
        template='presentation'
        )
fig.show()

Upon inspecting the charts, we observe a few things:
* General titles such as Data Analyst, Data Engineer, and Data Scientist are being used for all experience level
* Leadership indicator in titles such as Lead, Manager etc. are only present from mid-level or above.
* Titles that contains "Head" or "Director" are only present in SE or EX experience level. 

# Question 4<a id="q4"></a>

<div style="padding:20px;color:black;margin:0;font-size:200%;text-align:center;display:fill;border-radius:5px;background-color:#74bde0;overflow:hidden;font-weight:500">Does company location affect how much you can earn as a data scientist?</div>

To visualize this, I would need a choropleth map and I need the country ISO codes to use plotly's choropleth function. There's an easy way to do this, I'll scrape the ISO country codes table from the relevant Wikipedia page.

In [12]:
dfs = pd.read_html("https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes")

In [13]:
dfs[0].head(3)

ISO 3166[1]  \
                                    Country name[5]   
0                                       Afghanistan   
1  Akrotiri and Dhekelia – See United Kingdom, The.   
2                                     Åland Islands   

                                 Unnamed: 1_level_0  \
                             Official state name[6]   
0               The Islamic Republic of Afghanistan   
1  Akrotiri and Dhekelia – See United Kingdom, The.   
2                                             Åland   

                                 Unnamed: 2_level_0  \
                               Sovereignty[6][7][8]   
0                                   UN member state   
1  Akrotiri and Dhekelia – See United Kingdom, The.   
2                                           Finland   

                                       ISO 3166-1[2]  \
                                     Alpha-2 code[5]   
0  .mw-parser-output .monospaced{font-family:mono...   
1   Akrotiri and Dhekelia – See United Kingdom, The.   
2                                                 AX   

                                                     \
                                    Alpha-3 code[5]   
0                                               AFG   
1  Akrotiri and Dhekelia – See United Kingdom, The.   
2                                               ALA   

                                                     \
                                    Numeric code[5]   
0                                               004   
1  Akrotiri and Dhekelia – See United Kingdom, The.   
2                                               248   

                                      ISO 3166-2[3]  \
                          Subdivision code links[3]   
0                                     ISO 3166-2:AF   
1  Akrotiri and Dhekelia – See United Kingdom, The.   
2                                     ISO 3166-2:AX   

                                 Unnamed: 7_level_0  
                                  Internet ccTLD[9]  
0                                               .af  
1  Akrotiri and Dhekelia – See United Kingdom, The.  
2                                               .ax

In [14]:
dfs[0].columns = dfs[0].columns.droplevel()

In [15]:
dfs[0].columns

Index(['Country name[5]', 'Official state name[6]', 'Sovereignty[6][7][8]',
       'Alpha-2 code[5]', 'Alpha-3 code[5]', 'Numeric code[5]',
       'Subdivision code links[3]', 'Internet ccTLD[9]'],
      dtype='object')

In [16]:
iso_df = pd.DataFrame()
iso_df[['Country', 'alpha_2', 'iso_code']] = dfs[0][['Country name[5]', 'Alpha-2 code[5]', 'Alpha-3 code[5]']]

In [17]:
iso_df.head()

,Country,alpha_2,iso_code
0,Afghanistan,.mw-parser-output .monospaced{font-family:mono...,AFG
1,"Akrotiri and Dhekelia – See United Kingdom, The.","Akrotiri and Dhekelia – See United Kingdom, The.","Akrotiri and Dhekelia – See United Kingdom, The."
2,Åland Islands,AX,ALA
3,Albania,AL,ALB
4,Algeria,DZ,DZA


In [18]:
# merge the data with the dataframe containing iso country codes
FT_data = pd.merge(FT_data,
                  iso_df,# choose just these two columns
                  left_on='employee_residence',# the common column name
                  right_on='alpha_2', # common column name
                  how='left') # keep everything on FT_data only

In [19]:
FT_data.head(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,Country,alpha_2,iso_code
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L,Germany,DE,DEU
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S,Japan,JP,JPN
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M,United Kingdom of Great Britain and Northern I...,GB,GBR


In [20]:


def salary_aggregation(data: pd.DataFrame = FT_data, 
                       groupby_cols: list = ['employee_residence', 'iso_code'], 
                       value_cols: str or list = 'salary_in_usd',
                       agg_func= np.mean):
    """
    Return a groupby dataframe showing selected column based on an aggregation function
    :param data: the salary dataset, default: full-time only
    :param groupby_cols: list of columns to groupby with
    :param value_cols: list of columns to aggregate values of
    :param agg_func: an aggregate function to perform on selected value_cols, default: mean
    """
    # perform groupby aggregation
    agg_data = data.groupby(groupby_cols, as_index=False)[value_cols].agg(agg_func)
    
    # add a column to identify the aggregate function performed
    func_dict = {
        np.mean: 'mean',
        np.median: 'median',
        np.max: 'max',
        np.min: 'min',
        pd.Series.count: 'count'
    }
    
    for key, value in func_dict.items():
        if key == agg_func:
            agg_data['aggregation'] = func_dict[agg_func]
            break
    
    return agg_data


# initiate an empty list
data_agg_list = []
# append each aggregation df to the list
for func in [np.mean, np.median, np.max, np.min, pd.Series.count]:
    data_agg_list.append(salary_aggregation(agg_func=func))

# combine all df in the list, sort alphabetically
salary_agg_df = pd.concat(data_agg_list).sort_values('employee_residence')
salary_agg_df.head()   
    

,employee_residence,iso_code,salary_in_usd,aggregation
0,AE,ARE,100000.0,mean
0,AE,ARE,65000.0,min
0,AE,ARE,120000.0,max
0,AE,ARE,3.0,count
0,AE,ARE,115000.0,median


In [21]:
# create a choropleth map showing data scientist salary
fig = px.choropleth(salary_agg_df, locations='iso_code',
                    color='salary_in_usd', 
                    hover_name='iso_code',
                    color_continuous_scale='RdBu',
                    animation_frame='aggregation',# let viewer choose the aggregation function
                    scope='world',
                    title='Data Scientist Salary',
                    width=800,
                    height=800)

fig.show()

* Not much surprises here, developed countries typically have higher salary than developing countries. 
* The US has the best pay overall. 
* Most European countries don't actually pay their data scientists that well, a bit disappointing to see. 
* The salary in Malaysia is a bit high, could be an outlier, only one data point too. 
* The lowest salary in the US is less than 30,000 USD, this seems too low, also could be an outlier.  

In [22]:
# check data for that Malaysia's salary
FT_data[FT_data.iso_code=='MYS']

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,Country,alpha_2,iso_code
471,2022,SE,FT,Head of Data,200000,USD,200000,MY,100,US,M,Malaysia,MY,MYS


Apparently, there's only one person in the dataset that works in Malaysia, and they're Head of Data who works remotely. An outlier indeed. 

In [23]:
# check the lowest US salary
FT_data[(FT_data.salary_in_usd<30000)&(FT_data.iso_code=='USA')]

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,Country,alpha_2,iso_code
560,2022,SE,FT,Data Engineer,25000,USD,25000,US,100,US,M,United States of America (the),US,USA


It's remote work but it's a senior level experience and both the employee and company are in the US. This to me is an outlier too. However, is remote work simply paying less?

In [24]:
fig = px.histogram(FT_data,
                  x='salary_in_usd',
                  color='remote_ratio',
                  nbins=100,
                  barmode='overlay'
                  )
fig.show()

In [25]:
FT_data.remote_ratio = FT_data.remote_ratio.astype(str)
fig = px.box(FT_data,
                x='salary_in_usd',
                y='remote_ratio',
                color='remote_ratio',
                category_orders={'remote_ratio':['0', '50', '100']})
fig.show()

* Remote work don't necessarily pay less or more, it has a wider range of salaries, could be the nature of the work or company policy. There's not enough data to ascertain why some full-time remote work pays so little in this dataset. 

# Question 5<a id="q5"></a>

<div style="padding:20px;color:black;margin:0;font-size:200%;text-align:center;display:fill;border-radius:5px;background-color:#74bde0;overflow:hidden;font-weight:500">Are bigger companies paying more?</div>

In [26]:
fig = px.histogram(FT_data,
                  x='company_size', 
                  color='company_size',
                  category_orders={'company_size': ['S', 'M', 'L']}
                  )
fig.show()

In [27]:
FT_data.groupby('company_size')[['company_size', 'salary_in_usd']].mean()

,salary_in_usd
company_size,
L,119665.362694
M,118662.088050
S,76484.000000


In [28]:
fig = px.box(FT_data,
                x='salary_in_usd',
                y='company_size',
                color='company_size',
                category_orders={'company_size':['S', 'M', 'L']})
fig.show()

To answer the question: 
* company size do affect the salary being paid to data scientists but only if it's a small company.
* There's no significance difference between medium and large companies
* Although, large companies have more outliers. 

It's a little hard to judge the difference between medium and large companies,because there might be companies with 240 employees (i.e. medium size) and 251 employees (i.e. large) and naturally, they wouldn't be that different. 
